In [1]:
import pandas as pd
import io
import requests
import lxml
import numpy as np
import folium
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url)

In [17]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /Users/mac/miniconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1i             |       h35c211d_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.0 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1h-haf1e3a3_0 --> 1.1.1i-h35c211d_0



openssl-1.1.1i       | 1.9 MB    | ##################

In [2]:
print(len(dfs))
df = dfs[0]
df.info()

3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood  

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.     

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed   twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

- If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
# Dropping row within Borough that's not assigned
df['Borough'] =df['Borough'][~df.Borough.str.contains("Not assigned")]

In [4]:
# Group by postal code and combined neightbourhood if sharing same postal code
df = df.groupby(['Postal Code', 'Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)

In [5]:
# If neightbourhood not assigned, then assigned it with borough
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

In [6]:
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Import in the geospatial coordinate

In [7]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')

In [8]:
# join both table on postal 
df = df.join(df2.set_index('Postal Code'), on='Postal Code')

### Since we only want specific location which is Toronto downtown, We will only extract those within the coordinate

In [9]:
df = df.loc[df['Borough']== 'Downtown Toronto']

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [11]:
latitude = 	43.654260
longitude = -79.360636

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Downtown Toronto
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown Toronto',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.5
).add_to(venues_map)

# add the Borough as blue circle markers
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Borough']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# # display map
venues_map

# Clustering the Neighborhood 

In [13]:
# df.head()

In [14]:
# Toronto_cluser = df.transpose()
# Toronto_cluser.columns = ['Group-{}'.format(i) for i in range(0,len(Toronto_cluser.columns))]
# Toronto_cluser

## Seperation into 3 different cluster

In [15]:
# set number of clusters
kclusters = 3

manhattan_grouped_clustering = df.drop(['Postal Code', 'Borough','Neighbourhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

# Insert k cluster as column into df

df.insert(0, 'cluster label', kmeans.labels_)

# Mapping the cluster

In [16]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'],df['cluster label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# San Jose Food and entertaiment venues Exploration

In [104]:
# Hidden cell
CLIENT_ID = 'NVHSI1USRDZDFZ2P2FJTTKTDG343TF2K5M4YI1CASRQ2VX2N' # your Foursquare ID
CLIENT_SECRET = '3RNICWVFXC35EOGADJTN3LQBHLU0LHAAL4JQM1OPAWWVREO2' # your Foursquare Secret
ACCESS_TOKEN = 'L024RG4LPXZA4GEHDJ324JPSHGZRFWVL32XL5S4L2S5HNKDF' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NVHSI1USRDZDFZ2P2FJTTKTDG343TF2K5M4YI1CASRQ2VX2N
CLIENT_SECRET:3RNICWVFXC35EOGADJTN3LQBHLU0LHAAL4JQM1OPAWWVREO2


In [321]:
# Specify the location of our data point
address = 'San Jose, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Jose City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Jose City are 37.3361905, -121.890583.


In [322]:
# Generate a foursquare query for RESTAURANTS in the area
search_query = 'restaurant'
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)


# Get our result return from the foursqaure dbase
results = requests.get(url).json()
v=results['response']['venues']   # extract the result from json file
dframe = json_normalize(v)  # normalize our result with json normalize function

<ipython-input-322-9ae3c638c8e5>:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dframe = json_normalize(v)  # normalize our result with json normalize function


In [323]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

# dataframe_filtered

In [324]:
dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Valley Transportation Authority,Building,2 N 1st St Ste 100,at Santa Clara St,37.336388,-121.890705,"[{'label': 'display', 'lat': 37.33638837149467...",24,95113,US,San Jose,CA,United States,"[2 N 1st St Ste 100 (at Santa Clara St), San J...",NaN,4ccf30627b6854817136baf8
1,Cafe Eden,Mediterranean Restaurant,2 N 1st St,Santa Clara,37.336346,-121.890586,"[{'label': 'display', 'lat': 37.33634646299972...",17,95113,US,San Jose,CA,United States,"[2 N 1st St (Santa Clara), San Jose, CA 95113]",NaN,533479e1498e901ecf240ac8
2,VTA Santa Clara Light Rail Station,Light Rail Station,Fountain Alley,Btwn S 1st St & S 2nd St,37.335840,-121.890299,"[{'label': 'display', 'lat': 37.33583973154113...",46,95113,US,San Jose,CA,United States,"[Fountain Alley (Btwn S 1st St & S 2nd St), Sa...",NaN,4b991210f964a5205b5f35e3
3,Gente Cafe,Café,17 N 2nd St,NaN,37.337128,-121.889967,"[{'label': 'display', 'lat': 37.337128, 'lng':...",117,95113,US,San Jose,CA,United States,"[17 N 2nd St, San Jose, CA 95113]",NaN,5dbdeb65363a350007c28b24
4,VTA Downtown Customer Service Center,Office,55 W Santa Clara St Ste A,N 1st St,37.336087,-121.890467,"[{'label': 'display', 'lat': 37.33608663082123...",15,95113,US,San Jose,CA,United States,"[55 W Santa Clara St Ste A (N 1st St), San Jos...",NaN,4e6f803a45dd49e0f1ce93a1


In [325]:
dataframe_filtered['distance'] = dataframe_filtered['distance'].apply(lambda x: 'far' if x <= 53 else 'close')

In [326]:
df = dataframe_filtered[['name', 'categories','address','lat','lng', 'postalCode','formattedAddress', 'id','distance']]

In [327]:
df.head()

,name,categories,address,lat,lng,postalCode,formattedAddress,id,distance
0,Valley Transportation Authority,Building,2 N 1st St Ste 100,37.336388,-121.890705,95113,"[2 N 1st St Ste 100 (at Santa Clara St), San J...",4ccf30627b6854817136baf8,far
1,Cafe Eden,Mediterranean Restaurant,2 N 1st St,37.336346,-121.890586,95113,"[2 N 1st St (Santa Clara), San Jose, CA 95113]",533479e1498e901ecf240ac8,far
2,VTA Santa Clara Light Rail Station,Light Rail Station,Fountain Alley,37.335840,-121.890299,95113,"[Fountain Alley (Btwn S 1st St & S 2nd St), Sa...",4b991210f964a5205b5f35e3,far
3,Gente Cafe,Café,17 N 2nd St,37.337128,-121.889967,95113,"[17 N 2nd St, San Jose, CA 95113]",5dbdeb65363a350007c28b24,close
4,VTA Downtown Customer Service Center,Office,55 W Santa Clara St Ste A,37.336087,-121.890467,95113,"[55 W Santa Clara St Ste A (N 1st St), San Jos...",4e6f803a45dd49e0f1ce93a1,far


In [328]:
sanjose_map = folium.Map(location=[latitude, longitude], zoom_start=12)


incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df.lat, df.lng):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(df.lat)
longitudes = list(df.lng)
labels = list(df.categories)

for lat, lng, label in zip(df.lat,df.lng, df.categories):
    folium.Marker([lat, lng], popup=label).add_to(sanjose_map)    
    
# add incidents to map
sanjose_map.add_child(incidents)

In [329]:
df.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'postalCode',
       'formattedAddress', 'id', 'distance'],
      dtype='object')

In [330]:
# set number of clusters
kclusters = 2

group_cluster = df.drop(['name', 'categories','address', 'postalCode','formattedAddress','id','distance'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(group_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

# Insert k cluster as column into df

df.insert(0, 'cluster label', kmeans.labels_)

In [333]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['lat'], df['lng'], df['distance'],df['cluster label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [334]:
map_clusters